<a href="https://colab.research.google.com/github/limseo12/deep-learning-from-scratch_Study/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94%EB%94%A5%EB%9F%AC%EB%8B%9D3_%EC%A0%9C2%EA%B3%A0%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step11 가변 길이 인수(순전파 편)

In [ ]:
#11.1 Function 클래스 수정
class Function:
  def __call__(self, input):
    x = input.data    #1 Variable 이라는 '상자'에서 실제 데이터를 꺼낸다
    y = self.forward(x)   #2 forward 메서드에서 구체적인 계산을 한다
    output = Variable(as_array(y))    #3 계산 결과를 Variable에 넣고
    output.set_creator(self)    #4 자신이 '창조자'라고 원산지 표시?를 한다.
    self.input = input
    self.output = output
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

In [ ]:
#call 메서드의 인수와 반환값을 리스트로 바꿔보자.
class Function:
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    output = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs

  def forward(self, xs):
    raise NotImplementedError()
  
  def backward(self, gys):
    raise NotImplementedError()

In [ ]:
#11.2 Add 클래스 구현
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)

In [ ]:
xs = [Variable(np.array(2)), Variable(np.array(3))] #리스트로 준비
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

# Step 12 가변 길이 인수(개선 편)

In [ ]:
#12.1 첫 번째 개선 : 함수를 사용하기 쉽게
class Function:
  def __call__(self, *inputs):  # 별표를 붙인다.
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs

    #리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
    return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
>>> def f(*x):
print(x)

>>> f(1, 2, 3)
(1, 2, 3)

>>> f(1, 2, 3, 4, 5, 6)
(1, 2, 3, 4, 5, 6)

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

In [ ]:
# 12.2 두 번째 개선: 함수를 구현하기 쉽도록
class Function:
  def __call__(self, *inpuit):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)    # 1 별표를 붙여 언팩
    if not isinstance(ys, tuple):   # 2 튜플이 아닌 경우 추가 지원
      ys = (ys, )
    output = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.inputs = outputs

    return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

In [ ]:
#12.3 add 함수 구현
def add(x0, x1):
  return Add()(x0, x1)

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

# Step13 가변 길이 인수(역전파 편)